In [21]:
class SAES_cipher:
    def __init__(self, Key, IS, S, Me, IMe, GF):
#         self.plainText = plainText
        self.Key = Key
        self.IS = IS
        self.S = S
        self.Me = Me
        self.IMe = IMe
        self.GF = GF
    
    def rotNibble(self,w):
        return w[4:8]+w[:4]
    
    def Sboxes(self,bitss,sbx):
        idx = bitss[0]*2+bitss[1]*1
        jdx = bitss[2]*2+bitss[3]*1
        SB1 = sbx[idx][jdx]
        idx = bitss[4]*2+bitss[5]*1
        jdx = bitss[6]*2+bitss[7]*1
        SB2 = sbx[idx][jdx]
        SBL = []
        for item in SB1:
            SBL.append(item)
        for item in SB2:
            SBL.append(item)
        return SBL
        
    def keyEve(self,wx,some_const,wy):
        rot = self.rotNibble(wy)
        subs = self.Sboxes(rot,self.S)
        wz = []
        for i in range(len(wx)):
            wz.append(wx[i]^some_const[i]^subs[i])
        return wz
    
    def Exor(self,wx,wy):
        wz = []
        for i in range(len(wx)):
            wz.append(int(wx[i])^int(wy[i]))
        return wz
    
    def keyGen(self):
        w0, w1 = Key[:8], Key[8:16]
        w2 = self.keyEve(w0,[1,0,0,0,0,0,0,0],w1)
        w3 = self.Exor(w2,w1)
        w4 = self.keyEve(w2,[0,0,1,1,0,0,0,0],w3)
        w5 = self.Exor(w4,w3)
        K1 = w0+w1
        K2 = w2+w3
        K3 = w4+w5
        return K1,K2,K3
    
    def AddRoundKey(self,key,text):
        result = self.Exor(key,text)
        return result
    
    def shiftRows(self,text):
        return text[:4]+text[12:16]+text[8:12]+text[4:8]
    
    def GF_mul(self,x,y):
        i = 0
        j = 0
        for idx in range(4):
            i = i + x[3-idx]*(2**idx)
            j = j + y[3-idx]*(2**idx)
            
        answer = GF[i][j]
        answer = "{0:004b}".format(answer)
        return [int(item) for item in answer]
    
    def mixCol(self,text,M):
        state_mat = [ 
                    [text[:4],text[8:12]],
                    [text[4:8],text[12:16]]]

#         S = np.zeros((2,2))
#         for i in range(2):
#             for j in range(2):
#                 int_list = []
#                 for k in range(2):
#                     int_list.append(self.GF_mul(M[i][k],state_mat[k][j]))
#                 S[i][j] = self.Exor(int_list[0],int_list[1])
        
#         return S[0][0]+S[0][1]+S[1][0]+S[1][1]
                
        S00 = self.Exor(self.GF_mul(M[0][0],state_mat[0][0]),self.GF_mul(M[0][1],state_mat[1][0]))
        print('S00:',S00)
        S01 = self.Exor(self.GF_mul(M[0][0],state_mat[0][1]),self.GF_mul(M[0][1],state_mat[1][1]))
        print('S01:',S01)
        S10 = self.Exor(self.GF_mul(M[0][1],state_mat[0][0]),self.GF_mul(M[1][1],state_mat[1][0]))
        print('S10:',S10)
        S11 = self.Exor(self.GF_mul(M[0][1],state_mat[0][1]),self.GF_mul(M[1][1],state_mat[1][1]))
        print('S11:',S11)
        print()
        return S00+S10+S01+S11
    
    def roundN(self,key,text,num,tp):
        if tp == 'e':
            subs = self.Sboxes(text[:8],self.S)+self.Sboxes(text[8:16],self.S)
            print('Sboxes:',subs)
            sh_rows = self.shiftRows(subs)
            print('Shift Rows:',sh_rows)
            if num%2 == 0:
                mx_col = self.mixCol(sh_rows,self.Me)
                print('Mix Columns:',mx_col)
                return self.AddRoundKey(key,mx_col)
            else:
                return self.AddRoundKey(key,sh_rows)
        elif tp == 'd':
            sh_rows = self.shiftRows(text)
            print('Shift Rows:',sh_rows)
            subs = self.Sboxes(sh_rows[:8],self.IS)+self.Sboxes(sh_rows[8:16],self.IS)
            print('Sboxes:',subs)
            rnd_key = self.AddRoundKey(key,subs)
            print('Add Round Key:',rnd_key)
            
            if num%2 == 0:
                mx_col = self.mixCol(rnd_key,self.IMe)
                print('Mix Columns:',mx_col)
                return mx_col
            else:
                return rnd_key

In [22]:
text = 'sumitpatil'
plainText = []
for item in text:
    value = (ord(item)-97)%26
    plainText.append("{0:008b}".format(value))
    
sizeT = len(text)

In [23]:
import numpy as np
Key = [0,1,0,0,1,0,1,0,1,1,1,1,0,1,0,1]
IS = [[[1,0,1,0],[0,1,0,1],[1,0,0,1],[1,0,1,1]],
      [[0,0,0,1],[0,1,1,1],[1,0,0,0],[1,1,1,1]],
      [[0,1,1,0],[0,0,0,0],[0,0,1,0],[0,0,1,1]],
      [[1,1,0,0],[0,1,0,0],[1,1,0,1],[1,1,1,0]]]
S = [[[1,0,0,1],[0,1,0,0],[1,0,1,0],[1,0,1,1]],
      [[1,1,0,1],[0,0,0,1],[1,0,0,0],[0,1,0,1]],
      [[0,1,1,0],[0,0,1,0],[0,0,0,0],[0,0,1,1]],
      [[1,1,0,0],[1,1,1,0],[1,1,1,1],[0,1,1,1]]]
Me = [[[0,0,0,1],[0,1,0,0]],
      [[0,1,0,0],[0,0,0,1]]]
IMe = [[[1,0,0,1],[0,0,1,0]],
      [[0,0,1,0],[1,0,0,1]]]
GF = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
      [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],
      [0,2,4,6,8,10,12,14,3,1,7,5,11,9,15,13],
      [0,3,6,5,12,15,10,9,11,8,13,14,7,4,1,2],
      [0,4,8,12,3,7,11,15,6,2,14,10,5,1,13,9],
      [0,5,10,15,7,2,13,8,14,11,4,1,9,12,3,6],
      [0,6,12,10,11,13,7,1,5,3,9,15,14,8,2,4],
      [0,7,14,9,15,8,1,6,13,10,3,4,2,5,12,11],
      [0,8,3,11,6,14,5,13,12,4,14,7,10,2,9,1],
      [0,9,1,8,2,11,3,10,4,13,5,12,6,15,7,14],
      [0,10,7,13,14,4,9,3,14,5,8,2,1,11,6,12],
      [0,11,5,14,10,1,15,4,7,12,2,9,13,6,8,3],
      [0,12,11,7,5,9,14,2,10,6,1,13,15,3,4,8],
      [0,13,9,4,1,12,8,5,2,15,11,6,3,14,10,7],
      [0,14,15,1,13,3,2,12,9,7,6,8,4,10,11,5],
      [0,15,13,2,9,6,4,11,1,14,12,3,8,7,4,10]]

In [24]:
cipher = SAES_cipher(Key,IS,S,Me,IMe,GF)
k1,k2,k3 = cipher.keyGen()
print('Key 1:', k1)
print('Key 2:', k2)
print('Key 3:', k3)
print()

Key 1: [0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1]
Key 2: [1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0]
Key 3: [1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1]



In [26]:
cipherText = []
print('Message to be coded :', text)
print('Message in binary format :',plainText)
print()
i = 1
j = 0
while j < len(plainText):
    print('--------BLOCK',i,'----------')
    text = plainText[j:j+1]+plainText[j+1:j+2]
    j = j + 2
    i = i + 1
    print('----------ROUND1----------')
    P = []
    P[:0] = text[0]+text[1]
    new_P = cipher.AddRoundKey(k1,P)
    print('Before Round 1:',new_P)
    print()
    P_next = cipher.roundN(k2,new_P,0,'e')
    print('After Round 1:',P_next)

    # ROUND1
    print()
    print('----------ROUND2----------')
    cipherText.append(cipher.roundN(k3,P_next,3,'e'))
    
print('Cipher Text:',cipherText)


Message to be coded : ['00010010', '00010100']
Message in binary format : ['00010010', '00010100', '00001100', '00001000', '00010011', '00001111', '00000000', '00010011', '00001000', '00001011']

--------BLOCK 1 ----------
----------ROUND1----------
Before Round 1: [0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1]

Sboxes: [0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0]
Shift Rows: [0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0]
S00: [0, 0, 1, 0]
S01: [0, 1, 0, 0]
S10: [0, 0, 0, 0]
S11: [1, 1, 1, 1]

Mix Columns: [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1]
After Round 1: [1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1]

----------ROUND2----------
Sboxes: [0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1]
Shift Rows: [0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0]
--------BLOCK 2 ----------
----------ROUND1----------
Before Round 1: [0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1]

Sboxes: [1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0]
Shift Rows: [1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1

In [27]:
decipher = SAES_cipher(Key,IS,S,Me,IMe,GF)
k1,k2,k3 = decipher.keyGen()
print('Key 1:', k1)
print('Key 2:', k2)
print('Key 3:', k3)
print()

Key 1: [0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1]
Key 2: [1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0]
Key 3: [1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1]



In [30]:
decipherText = []
i = 0
j = 1
for item in cipherText:
    print()
    print('--------BLOCK',j,'----------')
    j = j + 1
    print('----------ROUND1----------')
    new_dP = decipher.AddRoundKey(k3,item)
    print('Before Round 1:',new_dP)
    print()
    dP_next = decipher.roundN(k2,new_dP,0,'d')
    print('After Round 2:',dP_next)

    # ROUND1
    print()
    print('----------ROUND2----------')
    decipherText.append(decipher.roundN(k1,dP_next,1,'d'))
    
print('Deciphered Text:',decipherText)



--------BLOCK 1 ----------
----------ROUND1----------
Before Round 1: [0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0]

Shift Rows: [0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1]
Sboxes: [1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1]
Add Round Key: [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1]
S00: [0, 0, 0, 1]
S01: [1, 1, 1, 1]
S10: [0, 1, 0, 0]
S11: [0, 1, 1, 0]

Mix Columns: [0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0]
After Round 2: [0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0]

----------ROUND2----------
Shift Rows: [0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0]
Sboxes: [0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1]
Add Round Key: [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0]

--------BLOCK 2 ----------
----------ROUND1----------
Before Round 1: [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0]

Shift Rows: [1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1]
Sboxes: [1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1]
Add Round Key: [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 

In [31]:
sblank = " "
i = 0
finalText = []
for item in decipherText:
    result = int("".join(str(i) for i in item),2)
    result = result + 97
    finalText.append(chr(result))

In [32]:
"".join(finalText)

'ት౩፰t\u086c'